In [2]:
!pip install mediapipe opencv-python

In [7]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [4]:
#VIDEO FEED
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame=cap.read() #Reads the next frame (image) from the camera. 
                          #frame = the image,
                          #ret = True/False (if frame was captured).
    cv2.imshow('Mediapipe Feed', frame) #Shows the webcam video in a window named "Mediapipe Feed".

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#### Make Detections

In [4]:
cap=cv2.VideoCapture(0)
#Setup mediapipe instance i.e. create the Pose Estimation model from MediaPipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#mp_pose.Pose(...)   ->    Loads the BlazePose model
    
#min_detection_confidence=0.5   ->   How sure the model must be to detect a person 
                                    #before tracking begins.
                                    #Range: 0 → 1 (higher = more accurate, slower)

#min_tracking_confidence=0.5   ->   How confidently the model must track the landmarks 
                                    #from frame to frame once detection starts.
                                    #Higher = smoother updates
#model name  ->  pose
    
    while cap.isOpened():
        ret, frame=cap.read() #Reads the next frame (image) from the camera. 
                          #frame = the image,
                          #ret = True/False (if frame was captured).


        #Detect stuff and render

        #Recolor image to RGB
        image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False  #Tells Python not to modify the image, which makes processing faster.
        
        #make detection
        results=pose.process(image) #finds body keypoints.

        #Recolor image back to BGR
        image.flags.writeable=True  #Allow the image to be modified again (for drawing landmarks)
        image=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)


        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                                                        

        #print(results)  #detection output (landmark coordinates)
        
        cv2.imshow('Mediapipe Feed', image) #Shows the webcam video in a window named "Mediapipe Feed".

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

#### Determine joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [8]:
cap=cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame=cap.read() 
        image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False 
        results=pose.process(image)
        image.flags.writeable=True  
        image=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks=results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass          
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                                                        
        cv2.imshow('Mediapipe Feed', image) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.812453628
y: 0.511454225
z: -0.968736172
visibility: 0.998993933
, x: 0.847050846
y: 0.454666018
z: -0.900154293
visibility: 0.997753322
, x: 0.864768863
y: 0.463330746
z: -0.900180519
visibility: 0.997639179
, x: 0.881955147
y: 0.472153932
z: -0.900580227
visibility: 0.997080624
, x: 0.79071188
y: 0.431924701
z: -0.935418904
visibility: 0.998684466
, x: 0.768327594
y: 0.425828934
z: -0.934979439
visibility: 0.999002397
, x: 0.745869398
y: 0.421543479
z: -0.935656428
visibility: 0.999115646
, x: 0.882967174
y: 0.519074202
z: -0.468272626
visibility: 0.996296942
, x: 0.686311
y: 0.456353545
z: -0.603862166
visibility: 0.999500394
, x: 0.827401
y: 0.602801561
z: -0.8017205
visibility: 0.998877704
, x: 0.759710312
y: 0.576214194
z: -0.840751052
visibility: 0.99962461
, x: 0.936729789
y: 0.899431348
z: -0.117898241
visibility: 0.986527383
, x: 0.46361962
y: 0.824738145
z: -0.529444695
visibility: 0.999077439
, x: 1.06528616
y: 1.40481043
z: 0.0165605675
visibility: 0.0288705379
, x: 

In [14]:
len(landmarks)

33

In [15]:
for i in mp_pose.PoseLandmark:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [16]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.832628429
y: 1.09031057
z: -0.372172594
visibility: 0.552174449

In [17]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value

11

In [19]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 1.18212545
y: 1.36850941
z: -0.808970928
visibility: 0.303531289

In [20]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.983727
y: 1.0883733
z: -0.885769069
visibility: 0.110841163

#### Calculate Angles

In [9]:
def calculate_angle(a,b,c):
    a=np.array(a) #First point
    b=np.array(b) #mid point
    c=np.array(c) #End point

    radians=np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    #Computes angle of the line BC with respect to horizontal. Then Computes angle of the line BA.
    #Difference gives the angle at B in radians.
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle

In [10]:
shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [11]:
calculate_angle(shoulder, elbow, wrist)

19.059182536046126

In [12]:
cap=cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame=cap.read() 
        image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False 
        results=pose.process(image)
        image.flags.writeable=True  
        image=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks=results.pose_landmarks.landmark

            #Get coordinates
            shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #Calculate angle
            angle=calculate_angle(shoulder, elbow, wrist)

            #Visualize angle
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )
        except:
            pass          
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                                                        
        cv2.imshow('Mediapipe Feed', image) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

#### Curl Counter

In [13]:
cap=cv2.VideoCapture(0)

#Curl counter variables
counter=0
stage=None


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame=cap.read() 
        image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False 
        results=pose.process(image)
        image.flags.writeable=True  
        image=cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        try:
            landmarks=results.pose_landmarks.landmark

            #Get coordinates
            shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #Calculate angle
            angle=calculate_angle(shoulder, elbow, wrist)

            #Visualize angle
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )

            #Curl Counter logic
            if angle>160:
                stage="down"
            if angle<30 and stage=="down":
                stage="up"
                counter+=1
                #print(counter)

                
        except:
            pass          

        #Render curl counter

        #Setup status box
        cv2.rectangle(image, (0,0), (225, 73), (245, 117, 16), -1)
        #(0,0) -> Top-left corner of the rectangle (x, y)
        #(225, 73) -> Bottom-right corner of the rectangle (x, y)
        #(245, 117, 16) -> Rectangle color in BGR format 
        #-1  ->   Fill the rectangle completely (if positive value like 2 → only border thickness)


        #Rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)


        #Stage data
        cv2.putText(image, 'STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        #render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                                                        
        cv2.imshow('Mediapipe Feed', image) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()